# Finetuning Ranker

Gratidão aos colegas Thiago Soares Laitz e Hugo (hugo@maritaca.ai) pelo apoio e código base fornecidos

# Installs

pip install bitsandbytes

pip install accelerate -U

pip install sentencepiece

!pip install transformers[torch]

!pip install datasets

## Neptune rastro

In [ ]:
import neptune.new as neptune 

In [ ]:
neptune.__version__

'0.16.18'

In [ ]:
import getpass


In [ ]:
os.environ['NEPTUNE_ALLOW_SELF_SIGNED_CERTIFICATE'] = 'TRUE'
os.environ['NEPTUNE_PROJECT'] = 'marcusborela/IA386DD'
os.environ['NEPTUNE_API_TOKEN'] = getpass.getpass('Informe NEPTUNE_API_TOKEN')

tag_contexto_rastro = 'INDIR_PTT5'
neptune_version = 0

In [ ]:
def converte_optimizer_state_dict(parm_optimizer)-> dict:
  """
    Recebe um objeto "parm_optimizer" que é do tipo "torch.optim.Optimizer" e retorna um dicionário 
    com informações sobre o otimizador.

    O dicionário de retorno é gerado a partir do estado do otimizador que é extraído da propriedade
    "state_dict()" do objeto "parm_optimizer", seu primeiro grupo de parâmetros do otimizador.
  """
  # return str(hparam['optimizer'])
  return parm_optimizer.state_dict()['param_groups'][0]
if neptune_version == 0:
  import neptune.new as neptune  
  class NeptuneRastroRun():
      se_geracao_rastro = True 
      neptune_project = ""
      tag_contexto_rastro = ""
      neptune_api_token = ""

      def __init__(self, parm_params:dict,  parm_lista_tag:list = None):
        # print(f"NeptuneRastroRun.init: se_geracao_rastro {self.__class__.se_geracao_rastro} parm_params `{parm_params} ")
        if self.__class__.se_geracao_rastro:      
          self.run_neptune = neptune.init(project=self.__class__.neptune_project, api_token=self.__class__.neptune_api_token, capture_hardware_metrics=True)
          self.run_neptune['sys/name'] = self.__class__.tag_contexto_rastro
          vparams = copy.deepcopy(parm_params)
          if "optimizer" in vparams:
            vparams["optimizer"] = converte_optimizer_state_dict(vparams["optimizer"])
          if 'criterion'  in vparams:
            vparams["criterion"] = str(vparams["criterion"])
          if 'scheduler'  in vparams:
            vparams["scheduler"] = str(type(vparams["scheduler"]))
          if 'device' in vparams:
            vparams['device'] = str(vparams["device"])
          self.device = vparams["device"]
          for tag in parm_lista_tag:
            self.run_neptune['sys/tags'].add(tag)
          self.run_neptune['parameters'] = vparams
          self.tmpDir = tempfile.mkdtemp()

      @property
      def run():
        return self.run_neptune

      @classmethod
      def ativa_geracao_rastro(cls):
        cls.se_geracao_rastro = True      

      @classmethod
      def def_contexto(cls):
        cls.se_geracao_rastro = True      

      @classmethod
      def desativa_geracao_rastro(cls):
        cls.se_geracao_rastro = False      

      @classmethod
      def retorna_status_geracao_rastro(cls):
        return cls.se_geracao_rastro      

      @classmethod
      def retorna_tag_contexto_rastro(cls):
        return cls.tag_contexto_rastro 

      @classmethod
      def inicia_contexto(cls, neptune_project, tag_contexto_rastro, neptune_api_token):
        assert '.' not in tag_contexto_rastro, "NeptuneRastroRun.init(): tag_contexto_rastro não pode possuir ponto, pois será usado para gravar nome de arquivo"      
        cls.neptune_api_token = neptune_api_token
        cls.tag_contexto_rastro = tag_contexto_rastro
        cls.neptune_project = neptune_project

      def salva_metrica(self, parm_metricas={}):
        #print(f"NeptuneRastroRun.salva_metrica: se_geracao_rastro {self.__class__.se_geracao_rastro} parm_metricas:{parm_metricas} ")
        if self.__class__.se_geracao_rastro:
          for metrica, valor in parm_metricas.items(): 
            self.run_neptune[metrica].log(valor)
  
      def gera_grafico_modelo(self, loader_train, model):
        if self.__class__.se_geracao_rastro: 
          # efetuar um forward 
          """
          se dataloader devolver x e y:
          """
          x_, y_ = next(iter(loader_train))
          x_ = x_.to(self.device)
          outputs = model(x_)
          """
          # se dataloader devolver dict:
          dados_ = next(iter(loader_train))
          outputs = model(dados_['x'].to(self.device))
          #outputs = model(x_['input_ids'].to(self.device), x_['attention_mask'].to(self.device))
          """
          nome_arquivo = os.path.join(self.tmpDir, "modelo "+ self.__class__.tag_contexto_rastro + time.strftime("%Y-%b-%d %H:%M:%S"))
          make_dot(outputs, params=dict(model.named_parameters()), show_attrs=True, show_saved=True).render(nome_arquivo, format="png")
          self.run_neptune["parameters/model_graph"].upload(nome_arquivo+'.png')
          self.run_neptune['parameters/model'] = re.sub('<bound method Module.state_dict of ', '',str(model.state_dict))      



      def stop(self):
        if self.__class__.se_geracao_rastro:         
          self.run_neptune.stop()

if neptune_version == 1:
  import neptune
  class NeptuneRastroRun():
      """
        Classe para geração de rastro de experimento utilizando a ferramenta Neptune.

        Busca implementar o rastro proposto em [Rastro-DM: Mineração de Dados com Rastro](https://revista.tcu.gov.br/ojs/index.php/RTCU/article/view/1664),
        autores Marcus Vinícius Borela de Castro e Remis Balaniuk, com o apoio da [solução Neptune](https://app.neptune.ai/)

        Attributes:
        -----------
        se_geracao_rastro : bool
            Indica se deve ser gerado rastro de experimento. 
        neptune_project : str
            Nome do projeto criado no Neptune. 
        tag_contexto_rastro : str
            Nome da tag utilizada para identificar o experimento.
        neptune_api_token : str
            Token utilizado para autenticação na API do Neptune. 
        run_neptune : object
            Objeto que representa o experimento no Neptune.
        device : str
            Dispositivo utilizado para o treinamento do modelo.
        tmpDir : str
          Diretório temporário utilizado para salvar gráfico do modelo.          
      """
      se_geracao_rastro = True 
      neptune_project = ""
      tag_contexto_rastro = ""
      neptune_api_token = ""

      def __init__(self, parm_params:dict,  parm_lista_tag:list = None):
        """
          Método construtor da classe NeptuneRastroRun.
          
          Args:
          - parm_params: dicionário contendo os parâmetros do modelo.
          - parm_lista_tag: lista contendo tags adicionais para o experimento.
        """      
        # print(f"NeptuneRastroRun.init: se_geracao_rastro {self.__class__.se_geracao_rastro} parm_params `{parm_params} ")
        if self.__class__.se_geracao_rastro:      
          self.run_neptune = neptune.init_run(project=self.__class__.neptune_project, api_token=self.__class__.neptune_api_token, capture_hardware_metrics=True)
          self.run_neptune['sys/name'] = self.__class__.tag_contexto_rastro
          vparams = copy.deepcopy(parm_params)
          if "optimizer" in vparams:
            vparams["optimizer"] = converte_optimizer_state_dict(vparams["optimizer"])
          if 'criterion'  in vparams:
            vparams["criterion"] = str(vparams["criterion"])
          if 'scheduler'  in vparams:
            vparams["scheduler"] = str(type(vparams["scheduler"]))
          if 'device' in vparams:
            vparams['device'] = str(vparams["device"])
          self.device = vparams["device"]
          for tag in parm_lista_tag:
            self.run_neptune['sys/tags'].add(tag)
          self.run_neptune['parameters'] = vparams
          # self.tmpDir = tempfile.mkdtemp()

      @property
      def run():
        """
        Retorna a instância do objeto run_neptune.
        """      
        return self.run_neptune

      @classmethod
      def ativa_geracao_rastro(cls):
        """
        Ativa a geração de rastro.
        """      
        cls.se_geracao_rastro = True      

      @classmethod
      def def_contexto(cls):
        """
        Define o contexto para a geração de rastro.
        """      
        cls.se_geracao_rastro = True      

      @classmethod
      def desativa_geracao_rastro(cls):
        """
        Desativa a geração de rastro.
        """      
        cls.se_geracao_rastro = False      

      @classmethod
      def retorna_status_geracao_rastro(cls):
        """
          Retorna o status da geração de rastro.
          
          Returns:
          - True se a geração de rastro está ativada, False caso contrário.
        """      
        return cls.se_geracao_rastro      

      @classmethod
      def retorna_tag_contexto_rastro(cls):
        """
          Retorna a tag do contexto de rastro.
        """      
        return cls.tag_contexto_rastro 

      @classmethod
      def inicia_contexto(cls, neptune_project, tag_contexto_rastro, neptune_api_token):
        """
        Inicia o contexto de execução no Neptune.

        Args:
            neptune_project (str): Nome do projeto no Neptune.
            tag_contexto_rastro (str): Tag que identifica o contexto de execução no Neptune.
            neptune_api_token (str): Token de acesso à API do Neptune.

        Raises:
            AssertionError: Caso a tag_contexto_rastro possua um ponto (.), 
              o que pode gerar erros na gravação de arquivo.
        """      
        assert '.' not in tag_contexto_rastro, "NeptuneRastroRun.init(): tag_contexto_rastro não pode possuir ponto, pois será usado para gravar nome de arquivo"      
        cls.neptune_api_token = neptune_api_token
        cls.tag_contexto_rastro = tag_contexto_rastro
        cls.neptune_project = neptune_project

      def salva_metrica(self, parm_metricas={}):
        """
          Salva as métricas no Neptune Run caso a geração de rastro esteja ativa.

          Parameters
          ----------
          parm_metricas: dict
              Dicionário contendo as métricas a serem salvas. As chaves devem ser os nomes das métricas e os valores devem ser
              os valores das métricas.
        """
        #print(f"NeptuneRastroRun.salva_metrica: se_geracao_rastro {self.__class__.se_geracao_rastro} parm_metricas:{parm_metricas} ")
        if self.__class__.se_geracao_rastro:
          for metrica, valor in parm_metricas.items(): 
            self.run_neptune[metrica].append(valor)
  
      def gera_grafico_modelo(self, loader_train, model):
        """
          Gera um gráfico do modelo e o envia para o Neptune. 
          Para gerar o gráfico, um forward pass é realizado em um batch de exemplos 
          de treino e o resultado é renderizado como um gráfico de nós conectados. 
          O gráfico é salvo em um arquivo .png e enviado para o Neptune como um arquivo anexo.

          Args:
              loader_train (torch.utils.data.DataLoader): DataLoader do conjunto de treinamento.
              model (torch.nn.Module): Modelo a ser visualizado.
          
          Pendente:
            Evolui para usar from io import StringIO (buffer = io.StringIO()) ao invés de tempdir 
        """    
        return

        """
        falta ajustar make_dot
        if self.__class__.se_geracao_rastro: 
          # efetuar um forward 
          batch = next(iter(loader_train))
          # falta generalizar linha abaixo. Criar função que recebe modelo e batch como parâmetro?
          outputs = model(input_ids=batch['input_ids'].to(hparam['device']), attention_mask=batch['attention_mask'].to(hparam['device']), token_type_ids=batch['token_type_ids'].to(hparam['device']), labels=batch['labels'].to(hparam['device']))
          nome_arquivo = os.path.join(self.tmpDir, "modelo "+ self.__class__.tag_contexto_rastro + time.strftime("%Y-%b-%d %H:%M:%S"))
          make_dot(outputs, params=dict(model.named_parameters()), show_attrs=True, show_saved=True).render(nome_arquivo, format="png")
          self.run_neptune["parameters/model_graph"].upload(nome_arquivo+'.png')
          self.run_neptune['parameters/model'] = re.sub('<bound method Module.state_dict of ', '',str(model.state_dict))      
        """


      def stop(self):
        """
          Para a execução do objeto Neptune. Todos os experimentos do Neptune são sincronizados com o servidor, e nenhum outro 
          experimento poderá ser adicionado a este objeto após a chamada a este método.
        """
        if self.__class__.se_geracao_rastro:         
          self.run_neptune.stop()

### Definindo parâmetros para o rastro


NeptuneRastroRun.inicia_contexto(os.environ['NEPTUNE_PROJECT'], tag_contexto_rastro,  os.environ['NEPTUNE_API_TOKEN'])
#NeptuneRastroRun.desativa_geracao_rastro()

In [ ]:
from transformers.integrations import NeptuneCallback

# Infra

## Paths

In [1]:
PATH_LOCAL_DATA = '../..'

In [5]:
PATH_TRAIN_MODEL_LOCAL = f"{PATH_LOCAL_DATA}/model/train/ptt5-base"

In [6]:
# path_data = '/content/drive/MyDrive/treinamento/202301_IA368DD/indir/data/train_data_juris_tcu_index_bm25.csv'

# PATH_TRAIN_DATA_ZIP = f"{PATH_LOCAL_DATA}/data/train_data_juris_tcu_index.zip"
PATH_TRAIN_DATA = f"{PATH_LOCAL_DATA}/data/train_juris_tcu_index/train_data_juris_tcu_index.csv"

In [7]:
os.path.exists(PATH_TRAIN_MODEL_LOCAL)

True

In [8]:
os.path.exists(PATH_TRAIN_DATA)

True

## Função de verificação de memória

In [9]:
from psutil import virtual_memory

In [10]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Wed Jul  5 19:28:08 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.116.03   Driver Version: 525.116.03   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:02:00.0 Off |                  N/A |
|  0%   48C    P8    26W / 370W |     58MiB / 24576MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [11]:
def mostra_memoria(lista_mem=['cpu']):
  """
  Esta função exibe informações de memória da CPU e/ou GPU, conforme parâmetros fornecidos.

  Parâmetros:
  -----------
  lista_mem : list, opcional
      Lista com strings 'cpu' e/ou 'gpu'.
      'cpu' - exibe informações de memória da CPU.
      'gpu' - exibe informações de memória da GPU (se disponível).
      O valor padrão é ['cpu'].

  Saída:
  -------
  A função não retorna nada, apenas exibe as informações na tela.

  Exemplo de uso:
  ---------------
  Para exibir informações de memória da CPU:
      mostra_memoria(['cpu'])

  Para exibir informações de memória da CPU e GPU:
      mostra_memoria(['cpu', 'gpu'])

  Autor: Marcus Vinícius Borela de Castro

  """
  if 'cpu' in lista_mem:
    vm = virtual_memory()
    ram={}
    ram['total']=round(vm.total / 1e9,2)
    ram['available']=round(virtual_memory().available / 1e9,2)
    # ram['percent']=round(virtual_memory().percent / 1e9,2)
    ram['used']=round(virtual_memory().used / 1e9,2)
    ram['free']=round(virtual_memory().free / 1e9,2)
    ram['active']=round(virtual_memory().active / 1e9,2)
    ram['inactive']=round(virtual_memory().inactive / 1e9,2)
    ram['buffers']=round(virtual_memory().buffers / 1e9,2)
    ram['cached']=round(virtual_memory().cached/1e9 ,2)
    print(f"Your runtime RAM in gb: \n total {ram['total']}\n available {ram['available']}\n used {ram['used']}\n free {ram['free']}\n cached {ram['cached']}\n buffers {ram['buffers']}")
    print('/nGPU')
    gpu_info = !nvidia-smi
  if 'gpu' in lista_mem:
    gpu_info = '\n'.join(gpu_info)
    if gpu_info.find('failed') >= 0:
      print('Not connected to a GPU')
    else:
      print(gpu_info)


In [12]:
mostra_memoria(['cpu','gpu'])

Your runtime RAM in gb: 
 total 67.35
 available 55.33
 used 10.97
 free 19.13
 cached 35.27
 buffers 1.97
/nGPU
Wed Jul  5 19:28:13 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.116.03   Driver Version: 525.116.03   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:02:00.0 Off |                  N/A |
|  0%   49C    P8    37W / 370W |     58MiB / 24576MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------

## Imports

In [13]:
import os

In [14]:
from transformers import Trainer
import numpy as np
import torch


/home/borela/miniconda3/envs/treinapython39/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [15]:
import pandas as pd

In [16]:
from dataclasses import dataclass, field

In [17]:
from datasets import Dataset

In [18]:
import json
from transformers import (
    AutoTokenizer,
    AutoConfig,
    MT5Tokenizer,
    AutoModelForSeq2SeqLM,
    AutoModelForSequenceClassification,
    Trainer,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    DataCollatorWithPadding,
    DataCollatorForSeq2Seq,
)
from tqdm.auto import tqdm
from datasets import Dataset
from dataclasses import dataclass, field


## Preparando para debug e display

In [19]:
def config_display():
  """
  Esta função configura as opções de display do Pandas.
  """

  # Configurando formato saída Pandas
  # define o número máximo de colunas que serão exibidas
  pd.options.display.max_columns = None

  # define a largura máxima de uma linha
  pd.options.display.width = 1000

  # define o número máximo de linhas que serão exibidas
  pd.options.display.max_rows = 100

  # define o número máximo de caracteres por coluna
  pd.options.display.max_colwidth = 50

  # se deve exibir o número de linhas e colunas de um DataFrame.
  pd.options.display.show_dimensions = True

  # número de dígitos após a vírgula decimal a serem exibidos para floats.
  pd.options.display.precision = 7


In [20]:
def config_debug():
  """
  Esta função configura as opções de debug do PyTorch e dos pacotes
  transformers e datasets.
  """

  # Define opções de impressão de tensores para o modo científico
  torch.set_printoptions(sci_mode=True)
  """
    Significa que valores muito grandes ou muito pequenos são mostrados em notação científica.
    Por exemplo, em vez de imprimir o número 0.0000012345 como 0.0000012345,
    ele seria impresso como 1.2345e-06. Isso é útil em situações em que os valores dos tensores
    envolvidos nas operações são muito grandes ou pequenos, e a notação científica permite
    uma melhor compreensão dos números envolvidos.
  """

  # Habilita detecção de anomalias no autograd do PyTorch
  torch.autograd.set_detect_anomaly(True)
  """
    Permite identificar operações que podem causar problemas de estabilidade numérica,
    como gradientes explodindo ou desaparecendo. Quando essa opção é ativada,
    o PyTorch verifica se há operações que geram valores NaN ou infinitos nos tensores
    envolvidos no cálculo do gradiente. Se for detectado um valor anômalo, o PyTorch
    interrompe a execução e gera uma exceção, permitindo que o erro seja corrigido
    antes que se torne um problema maior.

    É importante notar que a detecção de anomalias pode ter um impacto significativo
    no desempenho, especialmente em modelos grandes e complexos. Por esse motivo,
    ela deve ser usada com cautela e apenas para depuração.
  """

  # Configura variável de ambiente para habilitar a execução síncrona (bloqueante) das chamadas da API do CUDA.
  os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
  """
    o Python aguarda o término da execução de uma chamada da API do CUDA antes de executar a próxima chamada.
    Isso é útil para depurar erros no código que envolve operações na GPU, pois permite que o erro seja capturado
    no momento em que ocorre, e não depois de uma sequência de operações que pode tornar a origem do erro mais difícil de determinar.
    No entanto, é importante lembrar que esse modo de execução é significativamente mais lento do que a execução assíncrona,
    que é o comportamento padrão do CUDA. Por isso, é recomendado utilizar esse comando apenas em situações de depuração
    e removê-lo após a solução do problema.
  """

  # Define o nível de verbosity do pacote transformers para info
  # transformers.utils.logging.set_verbosity_info()


  """
    Define o nível de detalhamento das mensagens de log geradas pela biblioteca Hugging Face Transformers
    para o nível info. Isso significa que a biblioteca irá imprimir mensagens de log informativas sobre
    o andamento da execução, tais como tempo de execução, tamanho de batches, etc.

    Essas informações podem ser úteis para entender o que está acontecendo durante a execução da tarefa
    e auxiliar no processo de debug. É importante notar que, em alguns casos, a quantidade de informações
    geradas pode ser muito grande, o que pode afetar o desempenho do sistema e dificultar a visualização
    das informações relevantes. Por isso, é importante ajustar o nível de detalhamento de acordo com a
    necessidade de cada tarefa.

    Caso queira reduzir a quantidade de mensagens, comentar a linha acima e
      descomentar as duas linhas abaixo, para definir o nível de verbosity como error ou warning

    transformers.utils.logging.set_verbosity_error()
    transformers.utils.logging.set_verbosity_warning()
  """


  # Define o modo verbose do xmode, que é utilizado no debug
  # %xmode Verbose

  """
    Comando usado no Jupyter Notebook para controlar o modo de exibição das informações de exceções.
    O modo verbose é um modo detalhado que exibe informações adicionais ao imprimir as exceções.
    Ele inclui as informações de pilha de chamadas completa e valores de variáveis locais e globais
    no momento da exceção. Isso pode ser útil para depurar e encontrar a causa de exceções em seu código.
    Ao usar %xmode Verbose, as informações de exceção serão impressas com mais detalhes e informações adicionais serão incluídas.

    Caso queira desabilitar o modo verbose e utilizar o modo plain,
    comentar a linha acima e descomentar a linha abaixo:
    %xmode Plain
  """

  """
    Dica:
    1.  pdb (Python Debugger)
      Quando ocorre uma exceção em uma parte do código, o programa para a execução e exibe uma mensagem de erro
      com informações sobre a exceção, como a linha do código em que ocorreu o erro e o tipo da exceção.

      Se você estiver depurando o código e quiser examinar o estado das variáveis ​​e executar outras operações
      no momento em que a exceção ocorreu, pode usar o pdb (Python Debugger). Para isso, é preciso colocar o comando %debug
      logo após ocorrer a exceção. Isso fará com que o programa pare na linha em que ocorreu a exceção e abra o pdb,
      permitindo que você explore o estado das variáveis, examine a pilha de chamadas e execute outras operações para depurar o código.


    2. ipdb
      O ipdb é um depurador interativo para o Python que oferece recursos mais avançados do que o pdb,
      incluindo a capacidade de navegar pelo código fonte enquanto depura.

      Você pode começar a depurar seu código inserindo o comando ipdb.set_trace() em qualquer lugar do
      seu código onde deseja pausar a execução e começar a depurar. Quando a execução chegar nessa linha,
      o depurador entrará em ação, permitindo que você examine o estado atual do seu programa e execute
      comandos para investigar o comportamento.

      Durante a depuração, você pode usar comandos:
        next (para executar a próxima linha de código),
        step (para entrar em uma função chamada na próxima linha de código)
        continue (para continuar a execução normalmente até o próximo ponto de interrupção).

      Ao contrário do pdb, o ipdb é um depurador interativo que permite navegar pelo código fonte em que
      está trabalhando enquanto depura, permitindo que você inspecione variáveis, defina pontos de interrupção
      adicionais e até mesmo execute expressões Python no contexto do seu programa.
  """


In [21]:
config_display()

In [22]:
config_debug()

In [23]:
current_dir = os.getcwd()
print("Current directory:", current_dir)

Current directory: /home/borela/fontes/ind-ir/code/train


## Constants

In [24]:
TOKEN_FALSE = '▁não'
TOKEN_TRUE = '▁sim'

In [25]:
MODEL_NAME = 'unicamp-dl/ptt5-base-pt-msmarco-100k-v2'
# 'unicamp-dl/mt5-3B-mmarco-en-pt'

# Carga dos dados

In [27]:
df = pd.read_csv(PATH_TRAIN_DATA)

In [28]:
df.shape
# lim 100(111852, 6)

(402738, 7)

Verificando correção do arquivo!

In [29]:
print(df.isnull().sum())

QUERY_ID      0
DOC_ID        0
RELEVANCE     0
SCORE         0
TYPE          0
DOC_TEXT      0
QUERY_TEXT    0
Length: 7, dtype: int64


In [30]:
df[['QUERY_TEXT','DOC_TEXT']].applymap(len).describe()

,QUERY_TEXT,DOC_TEXT
count,402738.0000000,402738.0000000
mean,322.8252313,830.6957451
std,165.8299958,398.1844365
min,41.0000000,86.0000000
25%,217.0000000,572.0000000
50%,294.0000000,759.0000000
75%,391.0000000,1020.0000000
max,4212.0000000,3739.0000000


Para cada positivo, tem 5 negativos

In [31]:
df['RELEVANCE'].describe()

count    402738.0000000
mean          0.1666667
std           0.3726785
min           0.0000000
25%           0.0000000
50%           0.0000000
75%           0.0000000
max           1.0000000
Name: RELEVANCE, Length: 8, dtype: float64

In [32]:
df.head()

,QUERY_ID,DOC_ID,RELEVANCE,SCORE,TYPE,DOC_TEXT,QUERY_TEXT
0,151655,1943,1,0.897,TEMA,"O termo é ""Agente público"".\nAgente público te...",O dever de observância à hierarquia militar nã...
1,151655,15441,0,0.732,"relevant:TEMA, not relevant:TOTAL","O termo é ""Reforma-prêmio"".\nReforma-prêmio te...",O dever de observância à hierarquia militar nã...
2,151655,6373,0,0.717,"relevant:TEMA, not relevant:TOTAL","O termo é ""Exercício financeiro anterior"".\nEx...",O dever de observância à hierarquia militar nã...
3,151655,6973,0,0.680,"relevant:TEMA, not relevant:TOTAL","O termo é ""CJF"".\nCJF é classificado como uma ...",O dever de observância à hierarquia militar nã...
4,151655,7201,0,0.751,"relevant:TEMA, not relevant:TOTAL","O termo é ""Embratur"".\nEmbratur é classificado...",O dever de observância à hierarquia militar nã...


In [33]:
df["label"] = [TOKEN_FALSE if relevance == 0 else TOKEN_TRUE for relevance in df["RELEVANCE"]]

In [34]:
df.head()

,QUERY_ID,DOC_ID,RELEVANCE,SCORE,TYPE,DOC_TEXT,QUERY_TEXT,label
0,151655,1943,1,0.897,TEMA,"O termo é ""Agente público"".\nAgente público te...",O dever de observância à hierarquia militar nã...,▁sim
1,151655,15441,0,0.732,"relevant:TEMA, not relevant:TOTAL","O termo é ""Reforma-prêmio"".\nReforma-prêmio te...",O dever de observância à hierarquia militar nã...,▁não
2,151655,6373,0,0.717,"relevant:TEMA, not relevant:TOTAL","O termo é ""Exercício financeiro anterior"".\nEx...",O dever de observância à hierarquia militar nã...,▁não
3,151655,6973,0,0.680,"relevant:TEMA, not relevant:TOTAL","O termo é ""CJF"".\nCJF é classificado como uma ...",O dever de observância à hierarquia militar nã...,▁não
4,151655,7201,0,0.751,"relevant:TEMA, not relevant:TOTAL","O termo é ""Embratur"".\nEmbratur é classificado...",O dever de observância à hierarquia militar nã...,▁não


In [35]:
df.rename(columns={'DOC_TEXT': 'text', 'QUERY_TEXT':'query'},inplace=True)

In [36]:
df.head()

,QUERY_ID,DOC_ID,RELEVANCE,SCORE,TYPE,text,query,label
0,151655,1943,1,0.897,TEMA,"O termo é ""Agente público"".\nAgente público te...",O dever de observância à hierarquia militar nã...,▁sim
1,151655,15441,0,0.732,"relevant:TEMA, not relevant:TOTAL","O termo é ""Reforma-prêmio"".\nReforma-prêmio te...",O dever de observância à hierarquia militar nã...,▁não
2,151655,6373,0,0.717,"relevant:TEMA, not relevant:TOTAL","O termo é ""Exercício financeiro anterior"".\nEx...",O dever de observância à hierarquia militar nã...,▁não
3,151655,6973,0,0.680,"relevant:TEMA, not relevant:TOTAL","O termo é ""CJF"".\nCJF é classificado como uma ...",O dever de observância à hierarquia militar nã...,▁não
4,151655,7201,0,0.751,"relevant:TEMA, not relevant:TOTAL","O termo é ""Embratur"".\nEmbratur é classificado...",O dever de observância à hierarquia militar nã...,▁não


In [37]:
df = df[['query', 'text', 'label']]


In [38]:

df.shape

(402738, 3)

In [39]:
import gc

# ... código anterior ...

# Liberar memória utilizando gc.collect()
gc.collect()

20

# Separating evaluation data and prepare dataset tokenized

In [40]:
from sklearn.model_selection import train_test_split

In [43]:
train_df, valid_df = train_test_split(df, test_size=0.01,
                                      stratify=df['label'].values, random_state=123)
# Definir os argumentos de treinamento

In [44]:
train_df.shape, valid_df.shape

((398710, 3), (4028, 3))

In [45]:
valid_df[:2]

,query,text,label
270246,Verificado sobrepreço em contrato de obra públ...,"O termo é ""Fatura"".\nFatura tem nota de escopo...",▁sim
106266,Faculta-se aos médicos do poder judiciário o e...,"O termo é ""Médico"".\nMédico tem definição: ""Aq...",▁sim


In [46]:
print(np.unique(train_df['label'], return_counts=True), '\n', np.unique(valid_df['label'], return_counts=True))

(array(['▁não', '▁sim'], dtype=object), array([332258,  66452])) 
 (array(['▁não', '▁sim'], dtype=object), array([3357,  671]))


In [47]:
train_dataset = Dataset.from_pandas(train_df[["query", "text", "label"]].reset_index(drop=True))
valid_dataset = Dataset.from_pandas(valid_df[["query", "text", "label"]].reset_index(drop=True))

In [48]:
len(train_dataset), len(valid_dataset)

(398710, 4028)

In [49]:
valid_dataset[0]

{'query': 'Verificado sobrepreço em contrato de obra pública, a Administração deve promover ajuste do valor apurado nas faturas vincendas. Não existindo saldo financeiro, deve providenciar a instauração da competente tomada de contas especial.',
 'text': 'O termo é "Fatura".\nFatura tem nota de escopo: "É um documento onde está registrado um valor em debito com prazo de quitação da dívida seja parcelado ou à vista.".\nFatura tem termo relacionado: "Duplicata", "Nota fiscal eletrônica", "Nota fiscal" e "Cartão de crédito".\nFatura tem tradução em espanhol: "Factura".\nFatura tem tradução em inglês: "Invoice".',
 'label': '▁sim'}

In [50]:
del df, train_df, valid_df


In [51]:
gc.collect()

20

# Tokenize

In [52]:
# tokenizer = MT5Tokenizer.from_pretrained(MODEL_NAME)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

In [53]:
def tokenize(batch):
    queries_documents = [f"Query: {query} Document: {text} Relevant:" for query, text in zip(batch["query"], batch["text"])]
    print(f"Chamado tokenize len(queries_documents): {len(queries_documents)}")
    tokenized = tokenizer(
        queries_documents,
        padding=True, # "max_length",
        truncation=True,
        # return_tensors="pt",
        max_length= 512
    )
    # tokenized["labels"] = [[label] for label in batch["label"]]
    # tokenized['label'] = [[token_false, token_true][int(pairs["label"][i])]
    tokenized["labels"] = tokenizer(batch['label'])['input_ids']
    # tokenized["labels"] = [tokenizer.get_vocab()[token] for token in batch['label']]
    # tokenized["labels"] = [token_id_true if label == 'true' else token_id_false for label in batch["label"]]
    return tokenized


In [54]:
%%time
# valid_dataset.set_transform(tokenize)
valid_dataset = valid_dataset.map(
        tokenize,
        remove_columns=('query', 'text', 'label'),
        batched=True,
        desc='Tokenizing',
    )

Tokenizing:   0%|          | 0/4028 [00:00<?, ? examples/s]

Chamado tokenize len(queries_documents): 1000


Tokenizing:  50%|████▉     | 2000/4028 [00:00<00:00, 3247.28 examples/s]

Chamado tokenize len(queries_documents): 1000
Chamado tokenize len(queries_documents): 1000


Chamado tokenize len(queries_documents): 1000
Chamado tokenize len(queries_documents): 28
CPU times: user 3.68 s, sys: 138 ms, total: 3.82 s
Wall time: 1.06 s


In [55]:
%%time
# train_dataset.set_transform(tokenize)
train_dataset = train_dataset.map(
        tokenize,
        remove_columns=('query', 'text', 'label'),
        batched=True,
        desc='Tokenizing',
    )

Tokenizing:   0%|          | 1000/398710 [00:00<00:59, 6739.97 examples/s]

Chamado tokenize len(queries_documents): 1000
Chamado tokenize len(queries_documents): 1000


Tokenizing:   1%|          | 3000/398710 [00:00<00:56, 7036.97 examples/s]

Chamado tokenize len(queries_documents): 1000
Chamado tokenize len(queries_documents): 1000


Tokenizing:   1%|▏         | 5000/398710 [00:00<00:57, 6825.44 examples/s]

...

Tokenizing:  99%|█████████▉| 395000/398710 [00:58<00:00, 6467.16 examples/s]

Chamado tokenize len(queries_documents): 1000
Chamado tokenize len(queries_documents): 1000


Tokenizing: 100%|█████████▉| 397000/398710 [00:59<00:00, 6497.92 examples/s]

Chamado tokenize len(queries_documents): 1000
Chamado tokenize len(queries_documents): 1000


Chamado tokenize len(queries_documents): 710
CPU times: user 5min 27s, sys: 2.66 s, total: 5min 30s
Wall time: 59.3 s


In [56]:
print(valid_dataset[0])

{'input_ids': [2094, 540, 46, 1231, 6916, 81, 1044, 1164, 12, 1170, 4, 516, 1450, 3, 7, 5271, 697, 3072, 16630, 10, 941, 7, 8644, 53, 117, 1593, 1083, 6, 14591, 4575, 178, 5, 852, 18113, 17832, 6551, 3, 697, 17668, 33, 7, 14232, 1343, 93, 11, 18294, 4931, 4, 12558, 918, 5, 745, 797, 1806, 46, 28, 762, 21, 15, 3528, 1083, 64, 931, 1083, 87, 2649, 4, 19414, 46, 15, 3921, 16, 4064, 83, 141, 7578, 16, 941, 12, 4, 719, 123, 18, 4178, 4, 3366, 2424, 11, 9578, 474, 11084, 53, 52, 48, 1355, 5, 64, 931, 1083, 87, 762, 5993, 46, 15, 5787, 4152, 121, 49, 15, 1931, 121, 11408, 5441, 49, 15, 1931, 121, 11408, 27, 8, 15, 3022, 1380, 4, 8025, 64, 931, 1083, 87, 4101, 12, 1830, 46, 15, 3528, 167, 1083, 64, 931, 1083, 87, 4101, 12, 792, 46, 15, 2028, 601, 3249, 64, 294, 8985, 5572, 46, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

# Train

## setup

In [58]:
PATH_TRAIN_MODEL_LOCAL

'../../model/train/ptt5-base'

In [96]:
num_step_alert = 200
training_args = Seq2SeqTrainingArguments(output_dir=PATH_TRAIN_MODEL_LOCAL)
# Needed to make the Trainer work with an on-the-fly transformation on the dataset
# training_args.remove_unused_columns = False
training_args.output_dir = PATH_TRAIN_MODEL_LOCAL
training_args.warmup_steps=400 # Alterar!
training_args.num_train_epochs=4.0 # Alterar!
training_args.logging_steps=num_step_alert # Alterar!
training_args.save_strategy="steps"
training_args.save_steps=num_step_alert
training_args.save_total_limit=10
training_args.learning_rate=5e-5
training_args.per_device_train_batch_size=16 # t4: 8, a100-40: 32
training_args.gradient_accumulation_steps=4 # t4: 4, a100-40: 2
#training_args._n_gpu = 1
# training_args.bf16 = True # se for usar a100, 3090, 4090 -> usar
training_args.ignore_data_skip = True
training_args.load_best_model_at_end = True
training_args.evaluation_strategy='steps'
training_args.eval_steps=num_step_alert
training_args.do_eval = True
# training_args.optim='adamw_hf' #default
training_args.gradient_checkpointing = False # True
# se precisar economizar gpu
# training_args.optim='adamw_bnb_8bit'
# training_args.gradient_checkpointing = True
training_args.report_to="neptune",
# training_args.report_to = 'None'

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [59]:
!ls

drive  sample_data


In [97]:
print(training_args)

Seq2SeqTrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=200,
evaluation_strategy=steps,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_min_num_params=0,
fsdp_transformer_layer_cls_to_wrap=None,
full_determinism=False,
generation_max_length=None,
generation_num_beams=None,
gradient_accumulation_steps=4,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=False,
half_precision_backend=auto,
hub_model_id=None,
hub_private_repo=False,
hub_strategy=every_save,
hub_token=<HUB_TOKEN>,
ignore_data_skip=True,
in

In [61]:
# from transformers.integrations import NeptuneCallback
# rastro_neptune = NeptuneRastroRun(hparam, parm_lista_tag= tag_contexto_rastro)
# neptune_callback = NeptuneCallback(run=rastro_neptune)

In [71]:
# se local
nome_caminho_modelo = "/home/borela/fontes/relevar-busca/modelo/" + MODEL_NAME
assert os.path.exists(nome_caminho_modelo), f"Path para {MODEL_NAME} não existe!"


In [61]:
%%time
model = AutoModelForSeq2SeqLM.from_pretrained(nome_caminho_modelo)


CPU times: user 3.88 s, sys: 2.27 s, total: 6.15 s
Wall time: 1min 5s


In [74]:
trainer_cls = Seq2SeqTrainer
data_collator = DataCollatorForSeq2Seq(tokenizer)

In [106]:
# Limpa o cache da memória da GPU
# del trainer
torch.cuda.empty_cache()

In [107]:
gc.collect()

480

## Train

In [66]:
raise Exception('Parar aqui reinício!')

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 1>:1                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
Exception: Parar aqui reinício!

dicas para integrar com Neptune 
https://docs.neptune.ai/integrations/transformers/#__tabbed_2_2

In [108]:
trainer = trainer_cls(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

In [ ]:
del trainer

NameError: name 'trainer' is not defined

reiniciando checkpoint 5200

In [109]:
train_metrics = trainer.train(resume_from_checkpoint=True)

Loading model from ../../model/train/ptt5-base/checkpoint-5200.
You are resuming training from a checkpoint trained with 4.30.2 of Transformers but your current version is 4.25.1. This is not recommended and could yield to errors or unwanted behaviors.
/home/borela/miniconda3/envs/treinapython39/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 398710
  Num Epochs = 4
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 4
  Total optimization steps = 24920
  Number of trainable parameters = 222903552
  Continuing training from checkpoint, will skip to saved global_step
  Continuing training from epoch 0
  

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

  0%|          | 0/24920 [00:00<?, ?it/s]Didn't manage to set back the RNG states of the GPU because of the following error:
 RNG state is wrong size
This won't yield the same results as if the training had not been interrupted.
 22%|██▏       | 5400/24920 [14:04<22:45:30,  4.20s/it]***** Running Evaluation *****
  Num examples = 4028
  Batch size = 8


{'loss': 0.0386, 'learning_rate': 3.9804241435562806e-05, 'epoch': 0.03}


                                                       
 22%|██▏       | 5400/24920 [14:59<22:45:30,  4.20s/it]Saving model checkpoint to ../../model/train/ptt5-base/checkpoint-5400
Configuration saved in ../../model/train/ptt5-base/checkpoint-5400/config.json


...

Model weights saved in ../../model/train/ptt5-base/checkpoint-17800/pytorch_model.bin
tokenizer config file saved in ../../model/train/ptt5-base/checkpoint-17800/tokenizer_config.json
Special tokens file saved in ../../model/train/ptt5-base/checkpoint-17800/special_tokens_map.json
Copy vocab file to ../../model/train/ptt5-base/checkpoint-17800/spiece.model
Deleting older checkpoint [../../model/train/ptt5-base/checkpoint-15800] due to args.save_total_limit
 72%|███████▏  | 18000/24920 [15:31:11<8:02:44,  4.19s/it] ***** Running Evaluation *****
  Num examples = 4028
  Batch size = 8


{'loss': 0.0231, 'learning_rate': 1.4110929853181077e-05, 'epoch': 2.05}


                                                          
 72%|███████▏  | 18000/24920 [15:32:06<8:02:44,  4.19s/it]Saving model checkpoint to ../../model/train/ptt5-base/checkpoint-18000
Configuration saved in ../../model/train/ptt5-base/checkpoint-18000/config.json


{'eval_loss': 0.032688163220882416, 'eval_runtime': 55.4264, 'eval_samples_per_second': 72.673, 'eval_steps_per_second': 9.093, 'epoch': 2.05}


Model weights saved in ../../model/train/ptt5-base/checkpoint-18000/pytorch_model.bin
tokenizer config file saved in ../../model/train/ptt5-base/checkpoint-18000/tokenizer_config.json
Special tokens file saved in ../../model/train/ptt5-base/checkpoint-18000/special_tokens_map.json
Copy vocab file to ../../model/train/ptt5-base/checkpoint-18000/spiece.model
Deleting older checkpoint [../../model/train/ptt5-base/checkpoint-16000] due to args.save_total_limit
 73%|███████▎  | 18167/24920 [15:43:45<7:45:36,  4.14s/it] 

In [ ]:
train_metrics = trainer.train(resume_from_checkpoint=True)

In [ ]:
train_metrics = trainer.train(resume_from_checkpoint=False)

In [ ]:
EXECUÇÕES ANTERIORES

In [ ]:
%%time
train_metrics = trainer.train(resume_from_checkpoint=True)

In [ ]:
print(train_metrics )

Não sei se a mudança do batch size (32x2) 64 para 32 (8x4) impactou passar por dados duas vezes!!!

In [ ]:
%%time
train_metrics = trainer.train(resume_from_checkpoint=True)

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
train_metrics = trainer.train(resume_from_checkpoint=True)

Abaixo execucao lim50: a100/40gb (erro mount drive)

In [ ]:
%%time
train_metrics = trainer.train(resume_from_checkpoint=False)

In [ ]:
train_metrics = trainer.train(resume_from_checkpoint=True)

In [ ]:
%%time
train_metrics = trainer.train(resume_from_checkpoint=False)

In [ ]:
train_metrics = trainer.train(resume_from_checkpoint=True)

In [ ]:
train_metrics

In [ ]:
huggingface-cli login

In [ ]:
pip install huggingface_hub

In [ ]:
from huggingface_hub import notebook_login

In [ ]:
notebook_login()


In [ ]:
trainer.push_to_hub(
    model_id="ptt5-base-pt-msmarco-100k-v2-indir-lim100",
    repo_name="marcusborela"
    # use_auth_token=""
)